<a href="https://colab.research.google.com/github/EnisBerk/speech_audio_understanding/blob/master/notebooks/Audioset_model_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# GLOBAL parameters
# first run of notebook
first_run=True
data_required=True

In [0]:
if first_run==True:
  !pip install gputil

#   !pip install pysoundfile 
  #from mp3 to wav
#   !pip install pydub
  pass



In [0]:
# Import stuff
import numpy as np
# from pydub import AudioSegment
# from scipy.io.wavfile import read 

import IPython
import math
from google.colab import files as fs


import psutil
import humanize
import os
import GPUtil as GPU


In [0]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

In [0]:
if first_run==True:
  from google.colab import auth
  auth.authenticate_user()


In [0]:
if data_required==True:
  !gsutil cp gs://deep_learning_enis/speech_audio_understanding/data.tar.gz ./

  !tar xzf data.tar.gz

In [0]:
if data_required==True:

  !gsutil cp gs://deep_learning_enis/speech_audio_understanding/embeddings3.npy ./
  !gsutil cp gs://deep_learning_enis/speech_audio_understanding/postprocessed3.npy ./
  !gsutil cp gs://deep_learning_enis/speech_audio_understanding/data_file_indexes3.npy ./



In [0]:
embeddings=np.load('embeddings3.npy')
postprocessed=np.load('postprocessed3.npy')
data_file_indexes=np.load('data_file_indexes3.npy')
#get dict from numpy array
data_file_indexes=data_file_indexes[()]

In [0]:
mp3_data_files=sorted(list(data_file_indexes.keys()))

site_names=[]
hours=[]
for name in mp3_data_files:
  file_id=name
  name=name.split("_")
#   if len(name)!=18:
#     print(name)
  site_name=name[0]
  site_names.append(site_name)
  date=name[1]
  hour_min_sec=name[2]
  hour=hour_min_sec[0:2]
  hours.append(hour)
  year=date[0:4]
  month=date[4:6]
  day=date[6:8]
  site_id=name[2]

In [0]:
if data!=True:
  !curl https://storage.googleapis.com/deep_learning_enis/speech_audio_understanding/Eusipco2018_Google_AudioSet/md_50000_iters.tar --output ./md_50000_iters.tar
  !curl https://gist.githubusercontent.com/EnisBerk/19d817e9a6a9c060465a5b95c8d54d97/raw/8e0c24e92853e779e222bc3a152d7d65351cfe52/model.py --output ./model.py
  !curl https://raw.githubusercontent.com/qiuqiangkong/audioset_classification/master/metadata/class_labels_indices.csv --output ./class_labels_indices.csv

In [0]:
# LABELS 
import csv
labels=[]
with open('class_labels_indices.csv', newline='') as f:
    reader = csv.reader(f)
    headers=next(reader)
    for row in reader:
        print(row)
        labels.append(row[2])

['0', '/m/09x0r', 'Speech']
['1', '/m/05zppz', 'Male speech, man speaking']
['2', '/m/02zsn', 'Female speech, woman speaking']
['3', '/m/0ytgt', 'Child speech, kid speaking']
['4', '/m/01h8n0', 'Conversation']
['5', '/m/02qldy', 'Narration, monologue']
['6', '/m/0261r1', 'Babbling']
['7', '/m/0brhx', 'Speech synthesizer']
['8', '/m/07p6fty', 'Shout']
['9', '/m/07q4ntr', 'Bellow']
['10', '/m/07rwj3x', 'Whoop']
['11', '/m/07sr1lc', 'Yell']
['12', '/m/04gy_2', 'Battle cry']
['13', '/t/dd00135', 'Children shouting']
['14', '/m/03qc9zr', 'Screaming']
['15', '/m/02rtxlg', 'Whispering']
['16', '/m/01j3sz', 'Laughter']
['17', '/t/dd00001', 'Baby laughter']
['18', '/m/07r660_', 'Giggle']
['19', '/m/07s04w4', 'Snicker']
['20', '/m/07sq110', 'Belly laugh']
['21', '/m/07rgt08', 'Chuckle, chortle']
['22', '/m/0463cq4', 'Crying, sobbing']
['23', '/t/dd00002', 'Baby cry, infant cry']
['24', '/m/07qz6j3', 'Whimper']
['25', '/m/07qw_06', 'Wail, moan']
['26', '/m/07plz5l', 'Sigh']
['27', '/m/015lz1', 'S

In [0]:
from model import *

freq_bins = 128
classes_num = 527

# Hyper parameters
hidden_units = 1024
drop_rate = 0.5

model = FeatureLevelSingleAttention(
            freq_bins, classes_num, hidden_units, drop_rate)

In [0]:
checkpoint = torch.load("md_50000_iters.tar")
model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer'])
iteration = checkpoint['iteration']

model.eval()


FeatureLevelSingleAttention(
  (emb): EmbeddingLayers(
    (conv1): Conv2d(128, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv2): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv3): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (attention): Attention(
    (att): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1))
    (cla): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1))
  )
  (fc_final): Linear(in_features=1024, out_features=527, bias=True)
  (bn_attention): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [0]:
embeddings_tensor=torch.tensor(embeddings)
postprocessed_tensor = torch.tensor(postprocessed)
del embeddings,postprocessed

In [0]:
def inference(input_x,batch_size=512,first_k=1):
  predicted_labels=[]
  y_preds=torch.empty(0,first_k)
  y_pred_indexs=torch.empty((0,first_k),dtype=torch.long)
  start=0
  end=input_x.shape[0]-batch_size
  left=input_x.shape[0]%batch_size
  for i in range(0,end,batch_size):
    y_pred=model(input_x[i:i+batch_size].reshape(int(batch_size/10),10,128))
    y_pred, y_pred_index = torch.topk(y_pred, first_k, dim=1, largest=True, sorted=True)
    y_preds=torch.cat((y_preds,y_pred),0)
    y_pred_indexs=torch.cat((y_pred_indexs,y_pred_index),0)
    
  if left>0:
    y_pred=model(input_x[-left:].reshape(int(left/10),10,128))
    y_pred, y_pred_index = torch.topk(y_pred, first_k, dim=1, largest=True, sorted=True)
    y_preds=torch.cat((y_preds,y_pred),0)
    y_pred_indexs=torch.cat((y_pred_indexs,y_pred_index),0)
    
  return y_preds,y_pred_indexs

def filter_labels(y_preds,unwanted_labels,limit=10):
  predicted_labels=[]
  for sound_i in range(y_preds.shape[0]):
    sound_index=int(y_preds[sound_i].max(0)[1])
    label=labels[sound_index]
    predicted_labels.append(label)
    if label not in unwanted_labels and limit>0:
      limit-=1
      print(label)
      try:
        file_name="./data/split_00:10/" +str( mp3_data_files[i])[:-4]+".mp3"
        print(file_name)
        IPython.display.display(IPython.display.Audio(file_name))
      except:
        pass
    return predicted_labels

def filter_labels2(y_preds_index,unwanted_labels,limit=10):
  predicted_labels=set()
  for sound_i in y_preds_index:
    for m in sound_i[:1]:
      label=labels[int(m)]
      predicted_labels.add(label)
  return predicted_labels
    



In [0]:
printm()

Gen RAM Free: 601.4 MB  | Proc size: 12.5 GB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
try:
  del y_preds_post,y_preds_post_index
except:
  pass
try:  
  del y_preds_embed,y_preds_embed_index
except:
  pass

y_preds_post,y_preds_post_index=inference(postprocessed_tensor,batch_size=500,first_k=10)

# y_preds_embed,y_preds_embed_index=inference(embeddings_tensor,batch_size=500,first_k=10)



/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [0]:
def print_sample(y_preds,y_preds_index,count=2):
  for i,k in zip(y_preds_index[:count],y_preds[:count]):
    for m,n in zip(i,k):
      print(m,n)
    print("---")
    
print_sample(y_preds_post,y_preds_post_index)

tensor(173) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(171) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(166) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(179) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(174) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(175) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(176) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(177) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(178) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(526) tensor(4.1560e-39, grad_fn=<SelectBackward>)
---
tensor(173) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(171) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(166) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(179) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(174) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(175) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(176) tensor(4.1560e-39, grad_fn=<SelectBackward>)
tensor(177) tensor(4.1560e-

torch.Size([9550, 10])

In [0]:
labels[300]
y_preds_embed

'Vehicle'

In [0]:
# for i in y_preds_embed:
#   for k in i:
#     if k>1e-9:
#       print(k)

In [0]:
102//10

10

In [0]:
# unwanted_labels=["Speech", "Music","Vehicle","Field recording"]
# filter_labels2(y_preds_embed_index,unwanted_labels)
# all=[]
# print(y_preds_embed[0:5])
stats={}
files=[]
for i,embed in enumerate(y_preds_post_index):
  for i2,k in enumerate(embed[:2]):
    label=labels[k]
    if label=='Wood block':
#       print(mp3_data_files[i])
      files.append(((y_preds_post[i][i2]),mp3_data_files[i//10],i%10))
#     print(label)
    stats.setdefault(label,0)
    stats[label]+=1
#   print("-------")
print(stats)
# print(y_preds_embed_index[0:5])

# y_pred, y_pred_index = torch.topk(y_preds_embed, 10, dim=1, largest=True, sorted=True)
# print(y_pred[0:10])
# print(y_pred_index[0:10])

{'Wood block': 9550, 'Cymbal': 9550}


In [0]:
# unwanted_labels=["Speech", "Music","Vehicle","Field recording"]
# filter_labels2(y_preds_embed_index,unwanted_labels)
# all=[]
# print(y_preds_embed[0:5])
stats={}
files=[]
for i,embed in enumerate(y_preds_embed_index):
  for i2,k in enumerate(embed[:2]):
    label=labels[k]
    if label=='Vehicle':
#       print(mp3_data_files[i])
      files.append(((y_preds_embed[i][i2]),mp3_data_files[i//10],i%10))
#     print(label)
    stats.setdefault(label,0)
    stats[label]+=1
#   print("-------")
print(stats)
# print(y_preds_embed_index[0:5])

# y_pred, y_pred_index = torch.topk(y_preds_embed, 10, dim=1, largest=True, sorted=True)
# print(y_pred[0:10])
# print(y_pred_index[0:10])

{'Music': 9550, 'Speech': 7950, 'Vehicle': 1596, 'Bird': 4}


In [0]:
files.sort(reverse=True)
for file in files[:5]:
  filename2=file[1][:-4]+"_"+str(file[2])+".mp3"
  filename=file[1][:-4]+".mp3"
  print(filename2)
  print(file[0])
  filename="./data/split_00:10/"+filename
  IPython.display.display(IPython.display.Audio(filename))
#   fs.download(filename)

FSHCK4_20160629_194935_1140m_00s__1200m_00s_52m_00s__54m_00s_01m_00s__01m_10s_9.mp3
tensor(4.1560e-39, grad_fn=<SelectBackward>)


FSHCK4_20160629_194935_1140m_00s__1200m_00s_52m_00s__54m_00s_01m_00s__01m_10s_8.mp3
tensor(4.1560e-39, grad_fn=<SelectBackward>)


FSHCK4_20160629_194935_1140m_00s__1200m_00s_52m_00s__54m_00s_01m_00s__01m_10s_7.mp3
tensor(4.1560e-39, grad_fn=<SelectBackward>)


FSHCK4_20160629_194935_1140m_00s__1200m_00s_52m_00s__54m_00s_01m_00s__01m_10s_6.mp3
tensor(4.1560e-39, grad_fn=<SelectBackward>)


FSHCK4_20160629_194935_1140m_00s__1200m_00s_52m_00s__54m_00s_01m_00s__01m_10s_5.mp3
tensor(4.1560e-39, grad_fn=<SelectBackward>)


In [0]:
# stats={}
# for i in y_preds_post_index[0:100]:
#   for k in i[:2]:
#     print(labels[k])
#   print("-------")
print(y_preds_post[10:20])
print(y_preds_post_index[10:20])
labels[526]

tensor([[4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39,
         4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39,
         4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39,
         4.1560e-39, 4.1560e-39],
        [4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39,
         4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39,
         4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39,
         4.1560e-39, 4.1560e-39],
        [4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39,
         4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39,
         4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39,
         4.1560e-39, 4.1560e-39],
        [4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39,
         4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39, 4.1560e-39,
       

'Field recording'

In [0]:
for i in (y_preds_post_index):
  for k in (i):
    print (k)
  break


tensor(173)
tensor(171)
tensor(166)
tensor(179)
tensor(174)
tensor(175)
tensor(176)
tensor(177)
tensor(178)
tensor(526)


In [0]:
unwanted_labels=["Speech", "Music","Vehicle","Field recording"]

predicted_labels=filter_labels(y_preds_post,unwanted_labels,limit=10)
set(predicted_labels)

Snicker


{'Snicker'}

In [0]:
unwanted_labels=["Speech", "Music","Vehicle","Field recording"]

predicted_labels=filter_labels(y_preds_embed,unwanted_labels,limit=10)
set(predicted_labels)

In [0]:
_, pred = torch.topk(y_preds_embed, 5, dim=1, largest=True, sorted=False)

In [0]:
_, pred = torch.topk(y_preds_post, 4, dim=1, largest=True, sorted=False)

In [0]:
_[2]

tensor(4.1560e-39, grad_fn=<SelectBackward>)

In [0]:
known=torch.tensor([ 8,  6,  7, 19])
for k in pred:
  if False in k==known:
    print (k)

In [0]:
for k in [ 8,  6,  7, 19]:
  print(labels[k])

Shout
Babbling
Speech synthesizer
Snicker


In [0]:
# sample,time_stamp,freq
input1=(embeddings2[9].reshape(1,1,128))
input2=(embeddings2[9].reshape(1,1,128))
inputs=(embeddings2[:20].reshape(20,1,128))
# print(input1==input2)
# print(input1==inputs[9])
result1=model(input1)
result2=model(input2)
results=model(inputs)
print(result1==result2)
print(result1==results[9])
print(result1==results[9])

for i in range(result1.shape[1]):
  dif=float(result1[0][i])-float(results[9][i])
  dif=math.fabs(dif)
  if dif>0.000000001:
    print(dif)


In [0]:
result1.shape
results.shape

torch.Size([20, 527])

In [0]:
embeddings2=torch.tensor(embeddings)
embeddings2[9].reshape(1,1,128)==embeddings2[:20].reshape(20,1,128)[9]

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], dtype=torch.uint8)

In [0]:
asd=torch.ones(3,2,1)
print(asd.shape)
bcd=asd.transpose(1, 2)
bcd.shape

torch.Size([3, 2, 1])


torch.Size([3, 1, 2])

In [0]:
y_preds_post[0,0]-y_preds_post[0][19]

tensor(0., grad_fn=<SubBackward0>)

In [0]:
printm()

Gen RAM Free: 10.1 GB  | Proc size: 9.1 GB
GPU RAM Free: 11067MB | Used: 374MB | Util   3% | Total 11441MB
